In [43]:
import os
import pandas as pd 
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [44]:
base_dir = "submissions"

cv_df = pd.DataFrame()
name = []
cv = []

for filename in os.listdir(base_dir):
    if "oof" in filename:
        oof_df = pd.read_csv(os.path.join(base_dir, filename))
        cv_score = f1_score(oof_df["y"], oof_df["y_pred"], average="micro")
        trimmed_name = filename[:-8]
        name.append(trimmed_name)
        cv.append(cv_score)

cv_df["name"] = name
cv_df["cv"] = cv
cv_df = cv_df.sort_values(by=["cv"], ignore_index=True, ascending=False)
cv_df = cv_df[:5]
cv_df

,name,cv
0,2022_12_03-04:00:11,0.794594
1,2022_12_03-03:53:11,0.794594
2,2022_12_03-04:02:08,0.787687
3,2022_12_03-05:40:39,0.786504
4,2022_12_03-03:43:32,0.776243


In [45]:
oof = [pd.read_csv(os.path.join(base_dir, f"{name}.oof.csv")) for name in cv_df["name"]]
num_oof = len(oof)
num_pred = len(oof[0])
oof_preds = np.zeros((num_pred, num_oof))
oof_target = oof[0]["y"]
oof_cvs = [f1_score(oof_target, df["y_pred"], average="micro") for df in oof]

for i in range(num_oof):
    oof_preds[:, i] = oof[i]["y_pred"]

oof_preds.shape

(5068, 5)

In [52]:
model = AdaBoostClassifier()
scores = cross_val_score(model, oof_preds, oof_target, scoring="f1_micro", cv=5)
np.mean(scores)

0.7904505725372915

In [39]:
model = AdaBoostClassifier()
model = model.fit(oof_preds, oof_target)

In [41]:
subs = [pd.read_csv(os.path.join(base_dir, f"{name}.csv")) for name in cv_df["name"]]
num_test = len(subs[0])
num_sub = len(subs)
sub_pred = np.zeros((num_test, num_sub))

for i in range(num_sub):
    sub_pred[:, i] = subs[i]["y"]

sub_pred.shape

(3411, 5)

In [42]:
sub = pd.DataFrame()
sub["id"] = subs[0].id
sub["y"] = model.predict(sub_pred)
sub.to_csv("submissions/stacking.csv", index=False)